# Setting Up - Install Packages in the Workspace
Before you move on to the project instruction, follow the cells below to ensure you have the necessary packages installed. 
The cells below will also educate you on installing newer pinned packages, should you need them in the future. 

In [1]:
# Step 1. Update the PATH env var. 
import os
os.environ['PATH'] = f"{os.environ['PATH']}:/root/.local/bin"
os.environ['PATH'] = f"{os.environ['PATH']}:/opt/conda/lib/python3.6/site-packages"

In [ ]:
# Step 2. Restart the Kernel. 
# If you skip this step, your notebook may not be able to import the packages well. 


In [1]:
# Install the pinned version of packages, similar to below or use requirements.txt

!python -m pip install 'numpy==1.19.5' 'prompt-toolkit<2.0.0,>=1.0.15' 'jupyter-client>=7.0.0' 'jsonschema>=3.0.1' 'widgetsnbextension==3.0.0'


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-1.19.5.zip (7.3 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached widgetsnbextension-3.0.0-py2.py3-none-any.whl (2.5 MB)
  Using cached ipython-8.25.0-py3-none-any.whl (817 kB)
  Using cached traitlets-5.14.3-py3-none-any.whl (85 kB)
  Using cached ipython-8.24.0-py3-none-any.whl (816 kB)
  Using cached ipython-8.23.0-py3-none-any.whl (814 kB)


  Using cached ipython-8.22.2-py3-none-any.whl (811 kB)
  Using cached ipython-8.22.1-py3-none-any.whl (811 kB)
  Using cached ipython-8.21.0-py3-none-any.whl (810 kB)
  Using cached ipython-8.20.0-py3-none-any.whl (809 kB)
  Using cached ipython-8.18.1-py3-none-any.whl (808 kB)
  Using cached ipython-8.18.0-py3-none-any.whl (808 kB)
  Using cached ipython-8.17.2-py3-none-any.whl (808 kB)
  Using cached ipython-8.17.1-py3-none-any.whl (808 kB)
  Using cached ipython-8.16.1-py3-none-any.whl (806 kB)
  Using cached ipython-8.15.0-py3-none-any.whl (806 kB)
  Using cached ipython-8.14.0-py3-none-any.whl (798 kB)
  Using cached ipython-8.13.2-py3-none-any.whl (797 kB)
  Using cached ipython-8.13.1-py3-none-any.whl (797 kB)
  Using cached ipython-8.12.3-py3-none-any.whl (798 kB)
  Using cached ipython-8.12.2-py3-none-any.whl (797 kB)
  Using cached ipython-8.12.1-py3-none-any.whl (797 kB)
  Using cached ipython-8.11.0-py3-none-any.whl (793 kB)
  Using cached ipython-8.10.0-py3-none-any.whl (

  error: subprocess-exited-with-error
  
  × Building wheel for numpy (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [792 lines of output]
      setup.py:67: RuntimeWarning: NumPy 1.19.5 may not yet support Python 3.10.
        warnings.warn(
      Running from numpy source directory.
      numpy/random/_bounded_integers.pxd.in has not changed
      numpy/random/_pcg64.pyx has not changed
      numpy/random/_mt19937.pyx has not changed
      numpy/random/_bounded_integers.pyx.in has not changed
      Processing numpy/random/_bounded_integers.pyx
      numpy/random/_philox.pyx has not changed
      numpy/random/bit_generator.pyx has not changed
      numpy/random/_common.pyx has not changed
      numpy/random/_sfc64.pyx has not changed
      numpy/random/mtrand.pyx has not changed
      numpy/random/_generator.pyx has not changed
      Cythonizing sources
      blas_opt_info:
      blas_mkl_info:
      customize UnixCCompiler
        FOUND:
          libraries = ['mkl

In [2]:
# Check the version of any specific package
!python -m pip freeze | grep numpy

numpy @ file:///work/mkl/numpy_and_numpy_base_1682953417311/work


# Navigation

---

You are welcome to use this coding environment to train your agent for the project.  Follow the instructions below to get started!

### 1. Start the Environment

Run the next code cell to install a few packages.  This line will take a few minutes to run!

In [3]:
!pip -q install ./python

ERROR: Could not find a version that satisfies the requirement tensorflow==1.7.1 (from unityagents) (from versions: 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0, 2.12.1, 2.13.0rc0, 2.13.0rc1, 2.13.0rc2, 2.13.0, 2.13.1, 2.14.0rc0, 2.14.0rc1, 2.14.0, 2.14.1, 2.15.0rc0, 2.15.0rc1, 2.15.0, 2.15.0.post1, 2.15.1, 2.16.0rc0, 2.16.1)
ERROR: No matching distribution found for tensorflow==1.7.1


The environment is already saved in the Workspace and can be accessed at the file path provided below.  Please run the next code cell without making any changes.

In [4]:
from unityagents import UnityEnvironment
import numpy as np

# please do not modify the line below
env = UnityEnvironment(file_name="/data/Banana_Linux_NoVis/Banana.x86_64")

AttributeError: module 'collections' has no attribute 'MutableMapping'

Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [ ]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

Run the code cell below to print some information about the environment.

In [ ]:
# reset the environment
env_info = env.reset(train_mode=True)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Note that **in this coding environment, you will not be able to watch the agent while it is training**, and you should set `train_mode=True` to restart the environment.

In [ ]:
env_info = env.reset(train_mode=True)[brain_name] # reset the environment
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
while True:
    action = np.random.randint(action_size)        # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished
    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    if done:                                       # exit loop if episode finished
        break
    
print("Score: {}".format(score))

When finished, you can close the environment.

In [ ]:
env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  A few **important notes**:
- When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```
- To structure your work, you're welcome to work directly in this Jupyter notebook, or you might like to start over with a new file!  You can see the list of files in the workspace by clicking on **_Jupyter_** in the top left corner of the notebook.
- In this coding environment, you will not be able to watch the agent while it is training.  However, **_after training the agent_**, you can download the saved model weights to watch the agent on your own machine! 